# Separação de partições com Pytorch sobre o dataset MNIST

Complete os códigos onde for solicitado e faça 3 experimentos alterando o tamanho da divisão do dataset.

In [81]:
#importando  as bibliotecas
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import numpy as np

# Preparação dos dados

In [82]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Defina a proporção de divisão entre treinamento, validação e teste (float)
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

# Calcule o tamanho dos conjuntos de treinamento, validação e teste (int)
train_size = int(len(mnist_dataset) * train_ratio)
val_size = int(len(mnist_dataset) * val_ratio)
test_size = int(len(mnist_dataset) * test_ratio)

train_dataset, val_dataset, test_dataset = random_split(mnist_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Mostre o tamanho de cada conjunto


# Definição da rede

In [83]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [84]:
def train(model, train_loader, val_loader, epoch=1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(epoch):
        model.train()
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total
        print(f'Epoch {epoch + 1}, Validation Accuracy: {val_accuracy}')

In [85]:
def test(model, test_loader):
  model.eval()
  test_correct = 0
  test_total = 0
  with torch.no_grad():
      for data in test_loader:
          inputs, labels = data
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          test_total += labels.size(0)
          test_correct += (predicted == labels).sum().item()

  test_accuracy = test_correct / test_total
  print(f'Neural Network Test Accuracy: {test_accuracy}')

# Uso da rede

In [86]:
model = SimpleNN()

print("Treinando Rede Neural Simples:")
# Treine a rede neural simples

print(train(model, train_loader, val_loader, 5))


Treinando Rede Neural Simples:
Epoch 1, Validation Accuracy: 0.9302222222222222
Epoch 2, Validation Accuracy: 0.9482222222222222
Epoch 3, Validation Accuracy: 0.9513333333333334
Epoch 4, Validation Accuracy: 0.9604444444444444
Epoch 5, Validation Accuracy: 0.9665555555555555
None


In [87]:
# Avalie a rede neural simples
# Sua acurácia deve ser maior que 0.96
print(test(model, test_loader))

Neural Network Test Accuracy: 0.9618888888888889
None


Descreva aqui os 3 experimentos executados e quais foram os resultados obtidos em cada um deles:


**Primeiro experimento:**

train_ratio = 0.60
val_ratio = 0.25
test_ratio = 0.15

print(train(model, train_loader, val_loader, 6))


Treinando Rede Neural Simples:
Epoch 1, Validation Accuracy: 0.9210666666666667
Epoch 2, Validation Accuracy: 0.9468
Epoch 3, Validation Accuracy: 0.9516
Epoch 4, Validation Accuracy: 0.9499333333333333
Epoch 5, Validation Accuracy: 0.9572
Epoch 6, Validation Accuracy: 0.9645333333333334

Neural Network Test Accuracy: 0.9673333333333334


**Segundo experimento:**

train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

print(train(model, train_loader, val_loader, 6))

Treinando Rede Neural Simples:
Epoch 1, Validation Accuracy: 0.9205555555555556
Epoch 2, Validation Accuracy: 0.949
Epoch 3, Validation Accuracy: 0.9514444444444444
Epoch 4, Validation Accuracy: 0.9601111111111111
Epoch 5, Validation Accuracy: 0.9678888888888889
Epoch 6, Validation Accuracy: 0.9664444444444444


Neural Network Test Accuracy: 0.9666666666666667

**Terceiro experimento:**

train_ratio = 0.70
val_ratio = 0.10
test_ratio = 0.20

print(train(model, train_loader, val_loader, 4))

Treinando Rede Neural Simples:
Epoch 1, Validation Accuracy: 0.9251666666666667
Epoch 2, Validation Accuracy: 0.9415
Epoch 3, Validation Accuracy: 0.9553333333333334
Epoch 4, Validation Accuracy: 0.948


Neural Network Test Accuracy: 0.9501666666666667

# Métricas da rede

In [ ]:
from sklearn.metrics import confusion_matrix

# Criando uma matriz de confusão
y_pred = []
y_true = []

# Obtendo as previsões e rótulos reais no dataset
for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    y_pred.extend(predicted.tolist())
    y_true.extend(labels.tolist())

# Converção das listas para arrays NumPy
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Criando a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Imprimimindo a matriz de confusão
print(cm)

[[ 895    0    4    1    2    1    1    2    3    1]
 [   0 1009    8    0    3    0    0    3    3    1]
 [   3    5  827    2    4    0    1    5    9    0]
 [   2    7   13  865    1   12    1    7   16    6]
 [   0    2    1    0  898    0    5    2    2    3]
 [   6    3    5    6    1  747    5    1    8    3]
 [   7    1    5    0    7    4  846    0    7    0]
 [   1    3    4    0   11    0    0  948    1    8]
 [   3   10    4    5    6    7    1    1  789   10]
 [   1    3    0    6   26    5    0   13    3  833]]


In [97]:
for i in range(len(cm)):
  print(f'A precisão da classe {i+1}, foi de: {cm[i][i] / sum(cm[i])}')

A precisão da classe 1, foi de: 0.9835164835164835
A precisão da classe 2, foi de: 0.9824732229795521
A precisão da classe 3, foi de: 0.9661214953271028
A precisão da classe 4, foi de: 0.9301075268817204
A precisão da classe 5, foi de: 0.9835706462212487
A precisão da classe 6, foi de: 0.9515923566878981
A precisão da classe 7, foi de: 0.9646522234891676
A precisão da classe 8, foi de: 0.9713114754098361
A precisão da classe 9, foi de: 0.94377990430622
A precisão da classe 10, foi de: 0.9359550561797753


In [98]:
for i in range(len(cm)):
  print(f'Recall da classe {i+1}, foi de: {cm[i][i] / sum(cm[:, i])}')

Recall da classe 1, foi de: 0.9749455337690632
Recall da classe 2, foi de: 0.9674017257909875
Recall da classe 3, foi de: 0.9494833524684271
Recall da classe 4, foi de: 0.9774011299435028
Recall da classe 5, foi de: 0.9363920750782064
Recall da classe 6, foi de: 0.9626288659793815
Recall da classe 7, foi de: 0.9837209302325581
Recall da classe 8, foi de: 0.9653767820773931
Recall da classe 9, foi de: 0.9381688466111772
Recall da classe 10, foi de: 0.9630057803468208
